In [1]:
import numpy as np
import cv2
from openalpr import Alpr
import sys
import os
import shutil
import csv
from tqdm import tqdm

In [2]:
WINDOW_NAME  = 'openalpr'

alpr = Alpr('us', 'D:/openalpr_project/openalpr_64/openalpr.conf', 'D:/openalpr_project/openalpr_64/runtime_data')

if not alpr.is_loaded():
    print('Error loading OpenALPR')
    sys.exit(1)
    
alpr.set_top_n(3)

In [3]:
def get_max(result_dict):     
    
    if len(result_dict['confidence']) > 0:
        max_val = result_dict['confidence'][0]
        index = 0
    
        for i, conf in enumerate(result_dict['confidence']):
                if conf > max_val:
                    max_val = conf
                    index = i
                    
        return max_val, result_dict['plate'][index] 
    
    else:
        return 'No plate detected'

In [4]:
def get_license_plate(path_to_video, frame_skip = 1):
    cap = cv2.VideoCapture(path_to_video)
    
    if not cap.isOpened():
        cap.open(path_to_video)
        alpr.unload()
        sys.exit('Failed to open video file!')
        
    cv2.namedWindow(WINDOW_NAME, cv2.WINDOW_AUTOSIZE)
    cv2.setWindowTitle(WINDOW_NAME, 'OpenALPR video test')
    
    
    result_dict = {'confidence' : [], 'plate' : []}
    
    _frame_number = 0
    while True:
        ret_val, frame = cap.read()

        if not ret_val:
            break

        ret_val, enc = cv2.imencode("*.bmp", frame)

        _frame_number += 1

        if _frame_number % frame_skip != 0:
            continue

        #     cv2.imshow(WINDOW_NAME, frame)

        results = alpr.recognize_array(bytes(bytearray(enc)))

        if len(results['results']) > 0:
            result_dict['confidence'].append(results['results'][0]['confidence'])
            result_dict['plate'].append(results['results'][0]['plate'])

        if cv2.waitKey(1) == 27:
            break


#     for key, value in result_dict.items():
#         print(key, ' : ', value)
        
    return get_max(result_dict)

In [5]:
path_to_video = 'D:\openalpr_project\final_dir\plate'

get_license_plate(path_to_video)

SystemExit: Failed to open video file!

D:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3327: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [13]:
def create_csv(columns, path_csv):
    with open(path_csv, 'w') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(columns)

In [14]:
columns = ['video_file', 'plate', 'confidence', 'path']
path_csv = 'D://openalpr_project/data.csv'

# create_csv(columns, path_csv)

In [16]:
path_to_videos_dir = 'D://openalpr_project/videos/'
videos_files = os.listdir(path_to_videos_dir)

for video_file in tqdm(videos_files):
    path_video = os.path.join(path_to_videos_dir, video_file)
    
    if get_license_plate(path_video) == 'No plate detected':
        destination = 'D://openalpr_project/final_dir/no_plate/' + video_file
        shutil.copyfile(path_video, destination)
    else:
        destination = 'D://openalpr_project/final_dir/plate/' + video_file
        shutil.copyfile(path_video, destination)
        data_of_video = [video_file, get_license_plate(path_video)[1], get_license_plate(path_video)[0], path_video]
        
        with open(path_csv, 'a', newline = '') as csvfile:
            csvwriter = csv.writer(csvfile)
            csvwriter.writerow(data_of_video)
        

100%|██████████████████████████████████████████████████████████████████████████████| 281/281 [4:18:26<00:00, 55.18s/it]


In [ ]:
# while(cap.isOpened()):
#     ret, frame = cap.read()

#     gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

#     cv2.imshow('frame',gray)
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# cap.release()
# cv2.destroyAllWindows()

In [ ]:
cv2.destroyAllWindows()
cap.release()
alpr.unload()